In [6]:
from random import randint
from numpy import array
import numpy as np
from numpy import random
from keras.models import Model
from scipy.optimize import linprog
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam

%matplotlib inline

Using TensorFlow backend.


DATA

In [0]:
data = []
with open('/content/drive/My Drive/request_seq.txt' , errors='ignore' ) as fin:
    fin.seek(0)
    d = fin.read(100000000) 

In [8]:
#Data is continous sequence.Each segment in the sequence separated by '\n'
#Each segment contains 3 items: number of request, request ID and size of requested file.
d[:100]

'1 778030675 13539\n2 3484637698 17635\n3 1130267045 152\n4 3423744231 616\n5 203613471 121\n6 1342928892 '

In [9]:
#Spliting the continous sequence
data = d.split()
data[:10]

['1',
 '778030675',
 '13539',
 '2',
 '3484637698',
 '17635',
 '3',
 '1130267045',
 '152',
 '4']

In [10]:
del d
len(data)

12722261

In [11]:
#We are interested in number of request and request id.
req = []
for i in range(len(data)):
  r = []
  if (i-1)%3 == 0 and len(req)<2005000:
    r.append(data[i])
    #To convert string into real numbers. 
    temp = 0
    for j in range(len(r[0])):
      temp = temp + int(r[0][j])*pow(10,len(r[0])-j-1)
    req.append(temp)

req = np.array(req)
req.shape

(2005000,)

In [12]:
id, req1 = np.unique(req, return_inverse= True)
print(len(id))

763307


In [13]:
#To find the range of request id
min = np.min(req1)
max = np.max(req1)
min,max

(0, 763306)

In [14]:
# To reduce total numbers of ID
id = 100
factor = (max - min)/(id-1)
req = []
for i in range(len(req1)):
  req.append(int(np.divide(req1[i],factor)))
req = np.array(req)
req.shape

(2005000,)

In [0]:
del  req1,temp, data, r

In [16]:
count = np.zeros(id)
for i in range(1000):
    count[int(req[i])] += 1
count.shape

(100,)

In [0]:
prob = np.zeros((len(req)-1000,id), dtype = 'f4')

In [0]:
for i in range(len(req)-1000):
  count[int(req[i+1000])] += 1
  count[int(req[i])] -= 1
  prob[i] = count/1000

In [19]:
prob.shape

(2004000, 100)

In [0]:
del count

In [21]:
sample = 100
K = 10
encoder_input_data = prob[:2000000,:].reshape(sample,-1,id) # total samples = 100
batch_size = 20000
decoder_target_data = []
for i in range(1,sample + 1):
    temp = []
    for j in range(K): # to predict next 10 req for every batch
        temp.append(prob[int((batch_size*i) + j),:])
    decoder_target_data.append(temp)    
decoder_target_data = np.array(decoder_target_data).reshape(sample,K,id)

print(encoder_input_data.shape,decoder_target_data.shape)

(100, 20000, 100) (100, 10, 100)


In [22]:
decoder_input_data = np.zeros(decoder_target_data.shape)
decoder_input_data[1:,:,:] = decoder_target_data[:-1,:,:]
decoder_input_data[0,:,:] = encoder_input_data[-1,-1,:]
decoder_input_data.shape

(100, 10, 100)

MODEL

In [0]:
latent_dim = 128 # LSTM hidden units
dropout = 0.1

In [24]:
encoder_inputs = Input(shape=(None, id)) 
encoder = LSTM(latent_dim, dropout=dropout, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, id)) 
decoder_lstm = LSTM(latent_dim, dropout=dropout, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_outputs = Dense(id, activation='softmax')(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 100)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 100)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 117248      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  117248      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [25]:
model.compile(Adam(), loss='mean_squared_error')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=30)

Epoch 1/30
100/100 [==============================] - 64s 645ms/step - loss: 1.2796e-05
Epoch 2/30
100/100 [==============================] - 63s 627ms/step - loss: 1.2791e-05
Epoch 3/30
100/100 [==============================] - 63s 631ms/step - loss: 1.2782e-05
Epoch 4/30
100/100 [==============================] - 64s 637ms/step - loss: 1.2774e-05
Epoch 5/30
100/100 [==============================] - 63s 631ms/step - loss: 1.2765e-05
Epoch 6/30
100/100 [==============================] - 63s 631ms/step - loss: 1.2752e-05
Epoch 7/30
100/100 [==============================] - 63s 631ms/step - loss: 1.2744e-05
Epoch 8/30
100/100 [==============================] - 63s 631ms/step - loss: 1.2729e-05
Epoch 9/30
100/100 [==============================] - 66s 655ms/step - loss: 1.2717e-05
Epoch 10/30
100/100 [==============================] - 66s 658ms/step - loss: 1.2707e-05
Epoch 11/30
100/100 [==============================] - 64s 637ms/step - loss: 1.2693e-05
Epoch 12/30
100/100 [=========

FUTURE REQUEST PREDICTION

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)
pred_steps = K

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs = Dense(id, activation='softmax')(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, id))
    
    # Populate the first target sequence with end of encoding series pageviews
    target_seq[0, 0, :] = input_seq[0, -1, :]

    # Sampling loop for a batch of sequences - we will fill decoded_seq with predictions
    # (to simplify, here we assume a batch of size 1).

    decoded_seq = np.zeros((1,pred_steps,id))
    for i in range(pred_steps):
        
        output, h, c = decoder_model.predict([target_seq] + states_value)
        
        decoded_seq[0,i,:] = output[0,0,:]

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, id))
        target_seq[0, 0, :] = output[0,0,:]

        # Update states
        states_value = [h, c]

    return decoded_seq

In [35]:
n = int((len(prob) - batch_size)/5000)
n

396

In [36]:
prediction = []
for i in range(5000):
    if i%1000 == 0:
        print(i)
    prediction.append(decode_sequence(prob[i*n:i*n + batch_size,:].reshape(1,batch_size,id)))

0
1000
2000
3000
4000


In [37]:
prediction = np.array(prediction).reshape(5000,K,id)
prediction.shape

(5000, 10, 100)

GENERATION OF FAKE REQUESTS

In [41]:
fake_req = []
for i in range(5000):
    temp1 = []
    for j in range(K):
        temp = []
        for k in range(id):
             temp = (-prediction[i,j]).argsort()[:5]
        for k in range(5):
            temp1.append(temp[k]) 
    fake_req.append(temp1)        
fake_req = np.array(fake_req) 
fake_req.shape

(5000, 50)

In [53]:
len(req) + 5000*50

2255000

In [69]:
gen_req = []
for i in range(batch_size):
    gen_req.append(req[i])
for i in range(50):
    gen_req.append(fake_req[0][i])
for i in range(5000):
    for j in range(n + 51):
        if j <= n:
            gen_req.append(req[batch_size + j + n*i])
        elif(i>0):
            gen_req.append(fake_req[i-1,j-n-1])
gen_req = np.array(gen_req)
gen_req.shape

(2255000,)

In [70]:
gen_req = np.array(gen_req)
gen_req.shape

(2255000,)

In [71]:
req.shape

(2005000,)

In [0]:
np.save('req',req)
np.save('gen_req',gen_req)